In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, ConvLSTM2D, ReLU, concatenate, Subtract, Reshape, Input
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# Google 드라이브를 마운트하려면 이 셀을 실행하세요.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

In [0]:
@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [0]:
def load_image_test(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [0]:
def imgLoad(folder, shape, gray=True):
  im_dir = os.path.join("/content/drive/My Drive/workspace/Python/ML_Basic/Images", folder)
  x_train = []
  #x_train.append(np.zeros(shape, dtype=np.float32))
  
  file_list = os.listdir(im_dir)
  file_list = sorted(file_list, key=str.lower)
  for name in file_list:
      print(os.path.join(im_dir, name))
      if gray:
        im = cv2.imread(os.path.join(im_dir, name), 0)
        # plt.imshow(im)
        # plt.colorbar()
        # plt.grid(False)
        # plt.show()
      else:
        im = cv2.imread(os.path.join(im_dir, name))
        # plt.imshow(im)
        # plt.colorbar()
        # plt.grid(False)
        # plt.show()
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
      im = cv2.resize(im, dsize = shape[:2])
      im = im.astype(np.float32)
      x_train.append(im)

  # print(os.path.join(im_dir, file_list[0]))
  # im = cv2.imread(os.path.join(im_dir, file_list[0]), 0)
  # im = cv2.resize(im, dsize = (64, 64))
  # im = im.astype(np.float32)/255.
  # # im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  # x_train.append(im)

  # x_train = np.array(x_train)    
  return np.array(x_train)
  

In [7]:
# imgSet = imgLoad('ball_imgs')
shape = (128, 128, 3)
train_image = imgLoad('trainImg', shape, False)
train_mask = imgLoad('GroundTruth', shape, False)

/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/106024.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/153077.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/153093.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/181079.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/189080.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/208001.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/209070.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/21077.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/227092.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/24077.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/271008.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/304074.jpg
/content/drive/My Drive/workspace/Python/ML_Basic/Images/trainImg/326038.jpg
/

In [0]:
train_image, test_image = train_image[:90], train_image[90:]
train_mask, test_mask = train_mask[:90], train_mask[90:]

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_truth)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)